### Instalações e arquivos


In [ ]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 15.9 MB/s eta 0:00:00


In [ ]:
!pip install paramiko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 32.2 MB/s eta 0:00:00


In [ ]:
pip install sshtunnel

In [ ]:
import pandas as pd
import subprocess
import time
import pymongo
import paramiko
import json
from sshtunnel import SSHTunnelForwarder
from pymongo import MongoClient
from bson.objectid import ObjectId
import numpy as np


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
server = SSHTunnelForwarder(
    '',
    ssh_username="thais_gulias",
    ssh_pkey ="/content/thais_gulias",
    remote_bind_address=('', 27017),
    local_bind_address=('localhost', 27017)
)

server.start()

print(server.local_bind_port)  # show assigned local port
# tunel com ssh

27017


In [ ]:
# URI de conexão com o MongoDB
uri = ""

# Conectando ao MongoDB
try:
    client = pymongo.MongoClient(uri)
    print("Conexão com o MongoDB estabelecida com sucesso!")
except pymongo.errors.ConnectionFailure:
    print("Não foi possível conectar ao MongoDB. Verifique se o servidor está em execução.")

Conexão com o MongoDB estabelecida com sucesso!


In [ ]:
if client:
        # Acessar o banco de dados 'worflows'
        db = client ['workflows']


        # Acesse diretamente as coleções desejadas
workflows_collection = db['workflows']
workflowsitems_collection = db['workflowsItems']
marketplaces_collection = db['marketplaces']
scheduledbookings_collection = db['scheduledbookings']
schedules_collection = db['schedules']
workflowitemrelations_collection = db['workflowitemrelations']

### DADOS

In [ ]:
# Definir a query no workflowsitems_collection
workflow_id = ObjectId("teste")
query_items = { "workflowId": workflow_id }
workflow_items = workflowsitems_collection.find(query_items)

# Converter em DataFrame
workflow_items_df = pd.DataFrame(list(workflow_items))
workflow_items_df

FECHAR

In [ ]:
# fechar tunel SSH
server.stop()

In [ ]:
 # Fechar a conexão com o MongoDB
client.close()


In [ ]:
# Função para remover os últimos 9 caracteres do campo 'field'
def remove_last_9_chars(field_name):
    return field_name[:-9] if isinstance(field_name, str) else field_name

# Aplicar a transformação para remover os últimos 9 caracteres do 'field' para cada item dentro da lista de dicionários
for index, row in workflow_items_df.iterrows():
    workflow_items_df.at[index, 'data'] = [
        {'field': remove_last_9_chars(item['field']), 'value': item.get('value', None)} for item in row['data']
    ]

# Criar um DataFrame onde 'field' se torna colunas e 'value' são os valores dessas colunas
transformed_df = pd.DataFrame([
    {item['field']: item.get('value', None) for item in row}
    for row in workflow_items_df['data']
])

# Expandir o campo de endereço 'endereco_de_residencia_do_requerente' para várias colunas
if 'endereco_de_residencia_do_requerente' in transformed_df.columns:
    endereco_df = pd.json_normalize(transformed_df['endereco_de_residencia_do_requerente'])
    # Unir as novas colunas ao DataFrame original e remover a coluna aninhada original
    transformed_df = pd.concat([transformed_df.drop(columns=['endereco_de_residencia_do_requerente']), endereco_df], axis=1)


In [ ]:
# Substituir valores que representam NaN por np.nan
transformed_df.replace(['None', 'none', 'NULL', 'null', 'nan', ''], np.nan, inplace=True)
# Substitui NaN por uma string vazia
transformed_df.fillna('', inplace=True)


In [ ]:
# Lista de colunas a serem limpas
cols_to_clean = ['data_de_nascimento', 'data_de_nascimento_doa_estudante','data_de_nascimento_do_irmao']

# Aplicando a limpeza em cada coluna da lista com substituições encadeadas
for col in cols_to_clean:
    transformed_df[col] = (transformed_df[col]
                           .str.replace('T03:00:00.000Z', '', regex=False)
                           .str.replace('T02:00:00.000Z', '', regex=False))

In [ ]:
# Remover colchetes, chaves e aspas simples de todos os elementos do DataFrame, tratando valores nulos e entradas não-string
transformed_df = transformed_df.apply(lambda col: col.astype(str).str.replace('[', '', regex=False)
                                                        .str.replace(']', '', regex=False)
                                                        .str.replace('{', '', regex=False)
                                                        .str.replace('}', '', regex=False)
                                                        .str.replace("'", '') if col.dtype == 'object' else col)


In [ ]:
transformed_df['email_para_contato'] = transformed_df['email_para_contato'].str.replace('email: ', '', regex=False)

In [ ]:
# Filtrando as colunas que começam com o prefixo 'escolha_a_escola_destino'
cols_to_combine = transformed_df.filter(like='escolha_a_escola_destino')

# Concatenando as colunas em uma nova coluna, ignorando NaN e valores vazios
transformed_df['escolas_destino'] = cols_to_combine.apply(
    lambda x: ', '.join([val for val in x.dropna().astype(str) if val.strip()]), axis=1
)


In [ ]:
# Extraindo o bairro (a parte após o segundo traço)
transformed_df['bairro_escola'] = transformed_df['escolas_destino'].str.split(' - ').str[-1].str.strip()

In [ ]:
cols_to_combine_curso = transformed_df.filter(like='o_estudante_ira_cursar')

# Concatenando as colunas em uma nova coluna, ignorando NaN e valores vazios
transformed_df['ano_escolar_destino'] = cols_to_combine_curso.apply(
    lambda x: ', '.join([val for val in x.dropna().astype(str) if val.strip()]), axis=1
)

In [ ]:
cols_to_drop = transformed_df.filter(like='o_estudante_ira_cursar').columns.tolist() + \
               transformed_df.filter(like='escolha_a_escola_destino').columns.tolist()

# Removendo as colunas filtradas
transformed_df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
# Adicionando a coluna 'protocol' do workflow_items_df ao transformed_df
transformed_df['protocol'] = workflow_items_df['protocol']